<a href="https://colab.research.google.com/github/suajeong52/heaveMPM/blob/main/%EC%9C%A0%EC%A0%84%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_Termproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install smt

In [26]:
import pandas as pd
import numpy as np
import random
from smt.applications.mfk import MFK, NestedLHS

# 엑셀 파일 읽기
xH = pd.read_excel('xt_H_20.xlsx', usecols=[1]).values
xL = pd.read_excel('xt_L_20.xlsx', usecols=[1]).values
yH = pd.read_excel('mpm_re20_high.xlsx', usecols=[1]).values
yL = pd.read_excel('mpm_re20_low.xlsx', usecols=[1]).values
x_sort_low = pd.read_excel('pred_data_low_sort200.xlsx', usecols=[0]).values
y_pred_low = pd.read_excel('pred_data_low_sort200.xlsx', usecols=[1]).values


In [27]:
# Multi-Fidelity Kriging 모델 설정
sm = MFK(theta0=[1.0], corr='squar_exp', n_start=10)
sm.set_training_values(xL, yL)
sm.set_training_values(xH, yH, name=0)
sm.train()

# 예측 함수
def prediction_model(x):
    x_array = np.array(x).reshape(-1, 1)
    y_pred = sm.predict_values(x_array)
    return y_pred[0]

# 적합도 함수: Heave 운동을 최소화
def fitness(x):
    return -prediction_model(x)  # Heave 운동 값을 최소화하므로 음수로 반환


___________________________________________________________________________
   
                                    MFK
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 40
   
___________________________________________________________________________
   
 Training
   
   Training ...


/usr/local/lib/python3.10/dist-packages/smt/surrogate_models/krg_based.py:968: UserWarning: R is too ill conditioned. Poor combination of regression model and observations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/smt/surrogate_models/krg_based.py:968: UserWarning: R is too ill conditioned. Poor combination of regression model and observations.
  warnings.warn(


   Training - done. Time (sec):  0.6512113


In [29]:
import random

# 초기화
def initialize_population(pop_size, x_min, x_max):
    return np.random.uniform(x_min, x_max, (pop_size, 1))

# 선택 (룰렛 휠 선택)
def selection(population, fitnesses):
    total_fitness = np.sum(fitnesses)
    probabilities = fitnesses / total_fitness
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities.flatten())
    return population[selected_indices]

# 교배
def crossover(parent1, parent2, crossover_rate=0.8):
    if random.random() < crossover_rate:
        if len(parent1) > 1 and len(parent2) > 1:
            crossover_point = random.randint(1, len(parent1)-1)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            return child1, child2
    return parent1, parent2

# 돌연변이
def mutate(individual, mutation_rate=0.01, x_min=19, x_max=21):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = random.uniform(x_min, x_max)
    return individual

# 유전 알고리즘 실행
def genetic_algorithm(pop_size, x_min, x_max, generations, crossover_rate, mutation_rate):
    # 인구 크기를 짝수로 유지
    if pop_size % 2 != 0:
        pop_size += 1

    population = initialize_population(pop_size, x_min, x_max)
    for generation in range(generations):
        fitnesses = np.array([fitness(ind) for ind in population])
        population = selection(population, fitnesses)
        next_population = []
        for i in range(0, len(population) - 1, 2):  # 마지막 개체 제외
            parent1, parent2 = population[i], population[i+1]
            child1, child2 = crossover(parent1, parent2, crossover_rate)
            next_population.extend([mutate(child1, mutation_rate), mutate(child2, mutation_rate)])
        population = np.array(next_population)
    best_individual = population[np.argmax([fitness(ind) for ind in population])]
    return best_individual

# 파라미터 설정
pop_size = 50  # 인구 크기 짝수로 유지
x_min = 19
x_max = 21
generations = 100
crossover_rate = 0.8
mutation_rate = 0.01

# 유전 알고리즘 실행 및 최적화된 x 값 도출
best_x = genetic_algorithm(pop_size, x_min, x_max, generations, crossover_rate, mutation_rate)
print(f"최적화된 x 값: {best_x}")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
   Predicting ...
   Predicting - done. Time (sec):  0.0004821
   
   Prediction time/pt. (sec) :  0.0004821
   
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 1
   
   Predicting ...
   Predicting - done. Time (sec):  0.0005524
   
   Prediction time/pt. (sec) :  0.0005524
   
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 1
   
   Predicting ...
   Predicting - done. Time (sec):  0.0005789
   
   Prediction time/pt. (sec) :  0.0005789
   
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 1
   
   Predicting ...
   Predicting - done. Time (sec):  0.0005834
   
   Prediction time/pt. (sec) :  0.0005834
   
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 1
   
   Predictin